# Performance RNN
## Douglas Eck
This notebook shows you how to generate new performed compositions from a trained Performance RNN model. You'll see how to download a bundle containing a pre-trained model, instantiate and initialize the model and generate new polyphonic performances. The notebook also shows some hyperparameters useful for controlling generation, such as ``temperature``.

To hear the performances it is useful to have fluidsynth and pyfluidsynth installed. One way to install fluidsynth on OS X is via homebrew: ``brew install fluidsynth``.  One way to install pyfluidsynth is via pip: ``pip install pyfluidsynth``.

In [151]:
import os
import magenta
from magenta.models.drums_rnn import drums_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

import pandas as pd

from magenta.music.sequences_lib import concatenate_sequences

import magenta.music as mm

# Constants.
BUNDLE_DIR = '/home/jovyan/models/'
MODEL_NAME = 'drum_kit'
BUNDLE_NAME = 'drum_kit_rnn.mag'

In [3]:
mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))

In [43]:
generator_map = drums_rnn_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)


In [107]:
def compute_time(qpm=120, nsteps=16, steps_per_quarter=4):
    seconds_per_step = 60.0 / qpm / steps_per_quarter
    total_seconds = nsteps * seconds_per_step
    return total_seconds

In [230]:
def generate_one_bar_sequence(generator, qpm=120.0,temp=1.0,number_of_steps=16,beam_size=4, steps_per_quarter=4, primer=[]):
    generator_options = generator_pb2.GeneratorOptions()

    generator_options.args['temperature'].float_value = temp  # Higher is more random; 1.0 is default.
    generator_options.args['beam_size'].int_value = beam_size

    if len(primer) != 0:
        print("Using primer" + str(primer))
        primer_drums = magenta.music.DrumTrack([frozenset(pitches) for pitches in primer])
        primer_sequence = primer_drums.to_sequence(qpm=qpm)
    else:
        print("no primer")
        primer_sequence = music_pb2.NoteSequence()
        
    total_seconds = compute_time(qpm, number_of_steps, steps_per_quarter)
    last_end_time = compute_time(qpm, len(primer), steps_per_quarter)

    generate_section = generator_options.generate_sections.add(
        start_time=last_end_time,
        end_time=total_seconds)

    return generator.generate(primer_sequence, generator_options)

In [231]:
def adjust_sequence_times_and_merge(seq1, seq2, delta_time):
    """Adjusts note and total NoteSequence times by 'delta_time'."""
    retimed_seq2 = music_pb2.NoteSequence()
    retimed_seq2.CopyFrom(seq2)

    for note in retimed_seq2.notes:
        note.start_time += delta_time
        note.end_time += delta_time
        retimed_seq2.total_time += delta_time
    
    seq1.MergeFrom(retimed_seq2)
    
    return seq1 

In [283]:
generator.initialize()

qpm = 120.0
nsteps = 32
beams = 64

seqs = [
    generate_one_bar_sequence(generator=generator, qpm=qpm, temp=1.2 , number_of_steps=nsteps, beam_size=beams, primer = [(   ),(),(42 ,),(),(),(),(42, ),()]),
    generate_one_bar_sequence(generator=generator, qpm=qpm, temp=1.2, number_of_steps=nsteps, beam_size=beams, primer = [(   ),(),(42 ,),(),(67,),(),(42, ),()]),
    generate_one_bar_sequence(generator=generator, qpm=qpm, temp=1.3, number_of_steps=nsteps, beam_size=beams, primer = [(35,),(),(42 ,),(),(35,),(),(42, ),()]),
    generate_one_bar_sequence(generator=generator, qpm=qpm, temp=1.4 , number_of_steps=nsteps, beam_size=beams, primer = [(35,),(),(42 ,),(),(67,35),(),(42, ),()]),
    generate_one_bar_sequence(generator=generator, qpm=qpm, temp=1.5 , number_of_steps=nsteps, beam_size=beams, primer = [(35,),(),(42 ,),(),(67 ,35),(),(42, ),()])
]

Using primer[(), (), (42,), (), (), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -8.750489 
Using primer[(), (), (42,), (), (67,), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -13.973336 
Using primer[(35,), (), (42,), (), (35,), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -13.886360 
Using primer[(35,), (), (42,), (), (67, 35), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -20.431313 
Using primer[(35,), (), (42,), (), (67, 35), (), (42,), ()]
INFO:tensorflow:Beam search yields sequence with log-likelihood: -24.288216 


In [284]:
seq=seqs[0]
current_length=compute_time(qpm=qpm,nsteps=nsteps)
print(current_length)
for s in seqs[1:]:
    seq = adjust_sequence_times_and_merge(seq,s,current_length)
    current_length += compute_time(qpm=qpm,nsteps=nsteps)
    print(current_length)

4.0
8.0
12.0
16.0
20.0


In [285]:
#pd.DataFrame.from_records([dict(n.ListFields()) for n in list(seq.notes)])

In [286]:
#mm.plot_sequence(seq)
mm.play_sequence(seq, mm.midi_synth.fluidsynth)

In [22]:
from magenta.music.midi_io import sequence_proto_to_midi_file

sequence_proto_to_midi_file(sequence, output_file="performance_rnn_drums.midi")

In [23]:
sequence.SerializeToString

<function NoteSequence.SerializeToString>